In [2]:
%load_ext autoreload
%autoreload 2

import json5
import os
if "__file__" not in globals():
    __file__ = os.path.abspath(".")
print("Running in file:", __file__)
none_data_path=os.path.join(__file__, 'data','MatSciInstruct_none.json')
data_path=os.path.join(__file__,'data', 'MatSciInstruct.json')
print("Data path:", data_path)
print("None data path:", none_data_path)
with open(data_path, 'r') as f:
    # Load the JSON5 data from the file
    data= json5.load(f)
with open(none_data_path, 'r') as f:
    # Load the JSON5 data from the file
    none_data= json5.load(f)

print("Total number of data points:", len(data))
print("Total number of data points with None in 'answer':", len(none_data))

Running in file: /u01/mengpengyu/dataPipe
Data path: /u01/mengpengyu/dataPipe/data/MatSciInstruct.json
None data path: /u01/mengpengyu/dataPipe/data/MatSciInstruct_none.json
Total number of data points: 2307
Total number of data points with None in 'answer': 471


### 翻译部分问题，用于人工判别

In [ ]:
# from deep_translator import BaiduTranslator
# import json

# # 创建翻译器并进行翻译
# translator = BaiduTranslator(source='en', 
#                                   target='zh',
#                                   appid="20200702000511635",
#                                   appkey='QkuPegRc_zjsNH0Ytn52'
#                                   )
# # 翻译问题，存储到一个json文件中，后续方便查找
# translate_path=os.path.join(__file__, 'data','MatSciInstruct_translate.json')
# translate_list = []
# i=0
# for signal_data in data[:200]:
#     raw_problem = signal_data['metadata']['instruction']
#     translated_problem = translator.translate(raw_problem)
#     data_id = signal_data['id']
#     translate_list.append({
#         "id": data_id,
#         "raw_problem": raw_problem,
#         "translated_problem": translated_problem
#     })
#     if i % 10 == 0:
#         print(f"Translated {i} problems")
#     i+=1
# print("Translated problems:", translate_list)
# with open(translate_path, 'w') as f:
#     json.dump(translate_list, f, indent=4, ensure_ascii=False)

In [ ]:
# data[0:10]
translate_path=os.path.join(__file__, 'data','MatSciInstruct_translate.json')
with open(translate_path, 'r') as f:
    # Load the JSON5 data from the file
    translate_data= json5.load(f)
from model import call_openai,call_huoshan
prompt_path=os.path.join(__file__, 'prompts','division.txt')
with open(prompt_path, 'r') as f:
    raw_prompt = f.read()
end=5
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
for signal_data, trans_problem in zip(data[0:end], translate_data[0:end]):
    data_id = signal_data['id']
    input_data = {key: signal_data[key] for key in ['id', 'metadata']}
    prompt = raw_prompt.replace("{}",str(input_data))
    _ ,content=call_huoshan(prompt,'doubao')
    print(content)
    print(f"raw problem: {trans_problem['raw_problem']}")
    print(f"translated problem: {trans_problem['translated_problem']}")
    



{
  "id": "738abd95be987a0f02e0675d3f3387b1",
  "category": "KNOWLEDGE",
  "reason": "依据标准2.1，该问题要求从提供的上下文(input字段)中提取研究的推论，答案可直接从原文中找到。"
}
raw problem: What is the corollary of the study?
translated problem: 这项研究的推论是什么？


{
  "id": "5c65bef00762816a5d9de9ecf98b3bf2",
  "category": "KNOWLEDGE",
  "reason": "依据标准2.1，答案可以直接从提供的上下文(input字段)中提取找到关于x=0.2时Eu价态的具体描述。"
}
raw problem: What is the valence of Eu in EuPd$_2$(Si$_{1-x}$Ge$_x$)$_2$ for x = 0.2?
translated problem: 当x=0.2时，EuPd$_2$（Si$_{1-x}$Ge$_x$）$_2$中Eu的化合价是多少？


{
  "id": "f744adc51d773be28622ac121c2519b7",
  "category": "REASONING",
  "reason": "依据标准3.1，该问题要求对药物发现过程的多个阶段进行组织和归纳，形成连贯的概述，需要对信息进行处理和组织。"
}
raw problem: Write a brief overview of the drug discovery process.
translated problem: 简要概述药物发现过程。


{
  "id": "fb09b712b74a13f2fcae4405e22084fe",
  "category": "KNOWLEDGE",
  "reason": "依据标准2.1，该问题要求从提供的上下文(input字段)中提取引入子系统Arf不变量的动机，答案可直接概括得出。"
}
raw problem: What is the motivation for introducing the subsystem Arf invariant?
t